In [3]:
import pandas as pd
import os
from pandas import Series, DataFrame
df = pd.DataFrame()
sentlex = pd.read_csv('C:\\Users\\karis\\Desktop\\stanfordSentimentTreebank\\dictionary.txt', sep="|", names= ['phrase','phrase_ID'])
sentlex.head() ## phraseID and 



,phrase,phrase_ID
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [4]:
raw_score = pd.read_csv('C:\\Users\\karis\\Desktop\\stanfordSentimentTreebank\\sentiment_labels.txt', sep="|")
raw_score.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [14]:
raw_score = raw_score.rename(columns = {"phrase ids" : "phrase_ID", "sentiment values" : "sentiment_value"})
raw_score.head()


,phrase_ID,sentiment_value
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [8]:
# [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0]
# for very negative, negative, neutral, positive, very positive,
x = list(raw_score.sentiment_value)
from collections import OrderedDict
dic = OrderedDict()
for i in x:
    if (i >= 0 and i <= 0.2): 
        dic.update({i : 'very negative'})
    elif (i >0.2 and i <= 0.4):
        dic.update({i : 'negative'})
    elif (i >0.4 and i <= 0.6):
        dic.update({i : 'neutral'})
    elif (i >0.6 and i <= 0.8):
        dic.update({i : 'positive'})
    else:
        dic.update({i : 'very positive'})



    

In [11]:
rating_df = pd.DataFrame([
    [key,value] for key, value in dic.items()
])
rating_df = rating_df.rename(columns = {0 : "sentiment_value", 1 : "rating"})
rating_df.head()

,sentiment_value,rating
0,0.50000,neutral
1,0.44444,neutral
2,0.42708,neutral
3,0.37500,negative
4,0.41667,neutral


In [12]:
rating_merged = pd.merge(raw_score, rating_df, on=['sentiment_value'])
rating_merged.head()

,phrase_ID,sentiment_value,rating
0,0,0.5,neutral
1,1,0.5,neutral
2,3,0.5,neutral
3,17,0.5,neutral
4,18,0.5,neutral


In [17]:
sentlex_merged = pd.merge(sentlex, rating_merged, on=['phrase_ID'])
sentlex_merged.head()

,phrase,phrase_ID,sentiment_value,rating
0,!,0,0.50000,neutral
1,! ',22935,0.52778,neutral
2,! '',18235,0.50000,neutral
3,! Alas,179257,0.44444,neutral
4,! Brilliant,22936,0.86111,very positive


In [19]:
sentlex_merged.rating.value_counts()

neutral          119449
positive          50148
negative          43028
very positive     15255
very negative     11352
Name: rating, dtype: int64

In [36]:
import numpy as np
X = sentlex_merged.phrase
from sklearn.preprocessing import LabelEncoder
Label_encoded = LabelEncoder()
Y = Label_encoded.fit_transform(sentlex_merged['rating'].values)

print (X.shape)
print (Y.shape)
print (type(Y))

np.unique(Y)

(239232,)
(239232,)
<class 'numpy.ndarray'>


array([0, 1, 2, 3, 4], dtype=int64)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 3)

print (X_train.shape)
print (Y_train.shape)
print (X_test.shape)
print (Y_test.shape)


(167462,)
(167462,)
(71770,)
(71770,)


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()


# fit and transform X_train

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

LR = LogisticRegression()

LR.fit(X_train_dtm, Y_train)

Y_predicted = LR.predict(X_test_dtm)

print ('accracy:', accuracy_score(Y_test, Y_predicted))

accracy: 0.63500069667


In [44]:
def tokenize_test_SST_LR(vect):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    

    # fit and transform X_train

    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    LR = LogisticRegression()

    LR.fit(X_train_dtm, Y_train)

    Y_predicted = LR.predict(X_test_dtm)

    print ('accracy:', accuracy_score(Y_test, Y_predicted))

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tokenize_test_SST_LR(vect)

accracy: 0.63500069667


In [46]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test_SST_LR(vect)

accracy: 0.655817193814


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tokenize_test_SST_LR(vect)

accracy: 0.623686777205


In [48]:
def tokenize_test_SST_svm(vect):
    from sklearn import svm
    from sklearn.metrics import accuracy_score
    

    # fit and transform X_train

    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train_dtm, Y_train)
    Y_predicted = lin_clf.predict(X_test_dtm)

    print ('accracy:', accuracy_score(Y_test, Y_predicted))



In [49]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tokenize_test_SST_svm(vect)

accracy: 0.631879615438


In [51]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test_SST_svm(vect)

accracy: 0.640016720078


In [120]:
# split = pd.read_csv('C:\Users\karis\Desktop\stanfordSentimentTreebank\datasetSplit.txt', sep=",", header = None)
# sentence = pd.read_csv('C:\Users\karis\Desktop\stanfordSentimentTreebank\datasetSentences.txt', sep="\t", header = None)
# sos = pd.read_csv('C:\Users\karis\Desktop\stanfordSentimentTreebank\datasetSentences.txt', sep="\t", header = None)
# tree = pd.read_csv('C:\Users\karis\Desktop\stanfordSentimentTreebank\STree.txt', sep="\t", header = None)